In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch
import os
# *** Thêm thư viện nhận diện ngôn ngữ ***
from langdetect import detect, LangDetectException


print("Number of GPU: ", torch.cuda.device_count())
print("GPU Name: ", torch.cuda.get_device_name())


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)





g:\D\Anaconda\envs\cuda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of GPU:  1
GPU Name:  NVIDIA GeForce RTX 3070 Laptop GPU
Using device: cuda


In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",             # nf4: tốt nhất cho inference
    bnb_4bit_use_double_quant=True,        # giúp giảm lỗi số học
    bnb_4bit_compute_dtype=torch.float16   # giảm VRAM
)

In [3]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",  # Đưa model lên GPU
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:17<00:00,  4.40s/it]


In [4]:
# Detect lang and setup input

user_input = "Giới thiệu nội dung triết lý trong tác phẩm 'Chiếc thuyền ngoài xa'."
system_content_base = "You are an expert in analyzing and interpreting philosophical aspects in literature, with a focus on Vietnamese literary works."

detected_lang = "vi"

try:
    detected_lang = detect(user_input)
    print(f"Detected language: {detected_lang}")
except LangDetectException:
    print("Could not detect language, defaulting to English.")

response_language = "Vietnamese" if detected_lang == "vi" else "English"

system_content = f"{system_content_base} **Provide your response in {response_language}.**"
print(f"System Prompt: {system_content}")

Detected language: vi
System Prompt: You are an expert in analyzing and interpreting philosophical aspects in literature, with a focus on Vietnamese literary works. **Provide your response in Vietnamese.**


In [5]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
)

messages = [
    {"role": "system", "content": system_content},
    {"role": "user", "content": user_input},
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

outputs = model.generate(
    inputs,
    max_new_tokens=1024,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id, # Sử dụng eos_token_id làm pad_token_id
    eos_token_id=tokenizer.eos_token_id,
)

# Decode câu trả lời - Chỉ lấy phần mới được generate
response_start_index = inputs.shape[1]
decoded_output = tokenizer.decode(outputs[0][response_start_index:], skip_special_tokens=True)
print("\n--- Model Response ---")
print(decoded_output)
print("----------------------")

Device set to use cuda:0
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



--- Model Response ---
Tác phẩm "Chiếc thuyền ngoài xa" của Nguyễn Nhật Anh là một trong những tiểu thuyết được đánh giá cao về mặt triết lý và ý nghĩa văn hóa. Tác phẩm này khám phá những vấn đề triết lý quan trọng về cuộc sống, tử thần, tình yêu, trách nhiệm và trách nhiệm cá nhân.

Về mặt triết lý, "Chiếc thuyền ngoài xa" có thể được hiểu là một cuộc phiêu lưu tìm kiếm ý nghĩa của cuộc sống. Nhân vật chính, Hằng, là một cô gái trẻ sống trong một xã hội truyền thống, nhưng cô lại có một niềm tin mạnh mẽ về cuộc sống và tử thần. Cô cho rằng, cuộc sống là một chuyến phiêu lưu dài, và mỗi người đều phải tự tìm kiếm ý nghĩa của cuộc sống mình.

Tác phẩm cũng khám phá vấn đề về tử thần và trách nhiệm. Nhân vật chính phải đối mặt với tử thần của người thân, và phải tìm cách để vượt qua đau khổ và chấp nhận thực tế. Điều này có thể được hiểu là một cuộc đấu tranh để tìm kiếm ý nghĩa của cuộc sống, và chấp nhận rằng cuộc sống là ngắn ngủi và vô nghĩa.

Tình yêu cũng là một vấn đề triết lý q